# File to process data from survey among acquaintances
## from importing data to processing and visualizing

In [1]:
# Importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

## Importing data

In [2]:
# Importing data
df_whatsapp_data = pd.read_csv('data/umfrage-nachhaltige-mobilitaet_bekannte.csv')

## Methods for data processing

In [3]:
#Constants
age = 'Wie alt sind Sie? '
employment = 'Wie ist Ihre Beschäftigung? '
monthly_income = 'Bitte schätzen Sie das monatliche Netto Gesamteinkommen in Ihrem Haushalt in Euro. (Wenn Sie dazu keine Angaben machen möchten, überspringen Sie diese Frage bitte.)'
car_ownership = 'Besitzen Sie ein Auto? (auch Leasing) '
interest_e_car = 'Wie interessiert sind Sie an einem E-Auto als Ihr nächstes(/erstes) KFZ? (Wenn Sie generell kein Interesse am Besitz eines Autos haben, überspringen Sie diese Frage bitte) '
living_situation = 'Wie ist Ihre Wohnsituation? (Wohneigentum?) '
living_situation_house_apartment = 'Wie ist Ihre Wohnsituation? (Haus oder Wohnung?) '
living_situation_city = 'Wie ist Ihre Wohnsituation? (Großstadt, Stadt, Land?) '
mobility_car = 'Welchen Anteil an Ihrer Gesamtmobilität macht Ihr Auto aus? (falls Sie kein Auto besitzen, wählen Sie bitte 0% aus) '
mobility_public_transport = 'Welchen Anteil an Ihrer Gesamtmobilität macht der ÖPNV aus? '
mobility_public_transport_attraction = 'Was würde den ÖPNV für Sie attraktiver machen? '
mobility_e_car_attraction = 'Was würde E-Autos für Sie attraktiver machen? '

In [4]:
def explore_data(input_data):
    #Display a summary of the data
    # args: data: pandas dataframe
    # Summarize the data
    print(input_data.info())
    # Display the first 5 rows of the data
    print(input_data.head())
    
def drop_data(input_data):
    # Drop non-uniques and missing values
    # args: data: pandas dataframe
    # Drop duplicates
    duplicate_free_data = input_data.drop_duplicates()
    #data.drop_duplicates(inplace=True)
    # Drop missing values
    na_free_data = duplicate_free_data.dropna(inplace=True)
    # Replace missing values with the mean
    #data.fillna(data.mean(), inplace=True)
    return na_free_data

def fill_data(input_data):
    # Fill missing values with the mean
    # args: data: pandas dataframe
    #data.fillna(data.mean(), inplace=True)
    input_data.ffill().bfill()
    return input_data

# normalise age ranges to class representation
def normalise_age(input_data):
    # normalise age ranges to class representation
    # args: data: pandas dataframe
    # return: data: pandas dataframe
    input_data[age] = input_data[age].replace(['16-19', '20-25', '26-35', '36-45', '46-55', '56-65', '65+'], [17.5, 22.5, 30.5, 40.5, 50.5, 60.5, 65])
    return input_data

# remove euro sign from monthly income
def normalise_monthly_income(input_data):
    # remove euro sign from monthly income
    # args: data: pandas dataframe
    # return: data: pandas dataframe
    input_data[monthly_income] = input_data[monthly_income].replace('€', '', regex=True)
    input_data[monthly_income] = input_data[monthly_income].replace(' euro', '', regex=True)
    input_data[monthly_income] = input_data[monthly_income].replace(' Netto', '', regex=True)
    input_data[monthly_income] = input_data[monthly_income].replace('netto: ', '', regex=True)
    input_data[monthly_income] = input_data[monthly_income].replace(',', '.', regex=True)
    # convert to float for this input data
    input_data[monthly_income] = input_data[monthly_income].replace('Einkommen als SHK 300. Eltern 600', '900', regex=True)
    return input_data

# normalise car ownership to numerical representation
def normalise_car_ownership(input_data):
    # normalise car ownership to numerical representation
    # args: data: pandas dataframe
    # return: data: pandas dataframe
    input_data[car_ownership] = input_data[car_ownership].replace(['Auto', 'kein Auto'], [1, 0])
    return input_data

def split_data(input_data):
    # split data for use case
    # args: data: pandas dataframe
    # Split the data into features and target
    data_na_removed = input_data.dropna()
    selected_columns_finance = [age, employment, monthly_income, car_ownership, interest_e_car]#TODO check if these columns are relevant for correlation with financial situation
    selected_columns_mobility = [age, car_ownership, interest_e_car, mobility_car, mobility_public_transport, mobility_public_transport_attraction, mobility_e_car_attraction]#TODO check if these columns are relevant for correlation with mobility situation
    selected_columns_living = [living_situation, living_situation_house_apartment, living_situation_city, interest_e_car] #TODO check if these columns are relevant for correlation with living situation 
    selected_columns_age_employment = [age, employment, interest_e_car] #TODO check if these columns are relevant for correlation with age
    data_financial = input_data[selected_columns_finance]
    data_mobility = input_data[selected_columns_mobility]
    data_living = input_data[selected_columns_living]
    data_age = input_data[selected_columns_age_employment]
    return input_data, data_na_removed, data_financial, data_mobility, data_living, data_age

def prepare_data(input_data, feature_columns, target):
    # Prepare the data for analysis
    # args: data: pandas dataframe
    # args: feature_column: (list): liste der spalten die als feature genutzt werden sollen
    # args: target_column: (string): name zielvariable
    #return: tuple: X_train, X_test, y_train, y_test
    X_var = input_data[feature_columns]
    Y_var = input_data[target_column]
    # Split the data into training and testing data
    X_trains, X_tests, y_train, y_test = train_test_split(X_var, Y_var, test_size=0.2, random_state=42)
    return X_trains, X_tests, y_train, y_test

def scale_data(X_train_data, X_test_data):
    # Scale the data
    # args: X_train: pandas dataframe
    # args: X_test: pandas dataframe
    #return: X_train_scaled, X_test_scaled
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train_data)
    X_test_scaled = scaler.transform(X_test_data)
    return X_train_scaled, X_test_scaled

def train_model(X_train_data, y_train_data):
    # Train the model
    # args: X_train: pandas dataframe
    # args: y_train: pandas dataframe
    #return: model
    model = LinearRegression()
    model.fit(X_train_data, y_train_data)
    return model

def evaluate_model(model, X_test_data, y_test_data):
    # Evaluate the model
    # args: model: model
    # args: X_test: pandas dataframe
    # args: y_test: pandas dataframe
    #return: mse
    y_pred = model.predict(X_test_data)
    mse = mean_squared_error(y_test_data, y_pred)
    return mse

def analyze_data(X_train_data, X_test_data, Y_train_data, Y_test_data):
    # Analyze the data
    # args: X_train: pandas dataframe
    # args: X_test: pandas dataframe
    # args: Y_train: pandas dataframe
    # args: Y_test: pandas dataframe
    #return: mse
    # X_train_scaled, X_test_scaled = scale_data(X_train, X_test)
    # model = train_model(X_train_scaled, Y_train)
    # mse = evaluate_model(model, X_test_scaled, Y_test)
    # return mse
    model = LinearRegression()
    model.fit(X_train_data, Y_train_data)
    y_pred = model.predict(X_test_data)
    mse = mean_squared_error(Y_test_data, y_pred)
    print('Mean Squared Error:', mse)
    return mse

def visualize_data(input_data):
    # TODO Visualize the data with matplotlib and seaborn
    # args: data: pandas dataframe
    #return: None
    # TODO Implement the visualization
    print('Visualizing the data')
    #sns.pairplot(input_data)
    # visualize the regression line
    sns.lmplot(x=age, y=interest_e_car, data=input_data)
    # put title, labels and legend
    # plt.title('Pairplot of the data')
    # plt.xlabel('Values')
    # plt.ylabel('Frequency')
    # plt.legend()
    plt.show()
    
def visualize_correlation(input_data, feature_columns, target):
    # TODO Visualize the correlation between the features and the target
    # args: data: pandas dataframe
    # args: feature_columns: (list): liste der spalten die als feature genutzt werden sollen
    # args: target_column: (string): name zielvariable
    #return: None
    correlation_matrix = input_data[feature_columns + [target]].corr()
    plt.figure(figsize=(10, 10))
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5)
    plt.title('Correlation Matrix')
    plt.show()

def process_data(input_data, X_train_data, X_test_data, Y_train_data, Y_test_data, feature_columns, target):
    # Process the data
    # args: data: pandas dataframe
    #return: None
    explore_data(input_data)
    drop_data(input_data)
    fill_data(input_data)
    split_data(input_data)
    prepare_data(input_data, feature_columns, target)
    visualize_data(input_data)
    visualize_correlation(input_data, feature_columns, target)
    analyze_data(X_train_data, X_test_data, Y_train_data, Y_test_data)
    return None

In [ ]:
# method for linear regression analysis using scikit-learn
def linear_regression_analysis(input_data, feature_columns, target): 
    # Split the data
    sns.lmplot(x =age, y =interest_e_car, data = input_data, order = 2, ci = None)
    plt.show()
    input_data.ffill().bfill()
    x = np.array(input_data[age]).reshape(-1, 1) # TODO handle multiple features
    y = np.array(input_data[interest_e_car]).reshape(-1, 1) # TODO handle multiple targets
 
    # Separating the data into independent and dependent variables
    # Converting each dataframe into a numpy array 
    # since each dataframe contains only one column
    input_data.dropna(inplace = True)
 
    # Dropping any rows with Nan values
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)
 
    # Splitting the data into training and testing data
    regr = LinearRegression()
 
    regr.fit(x_train, y_train)
    print(regr.score(x_test, y_test))
    y_pred = regr.predict(x_test)
    plt.scatter(x_test, y_test, color ='b')
    plt.plot(x_test, y_pred, color ='k')
 
    plt.show()
    # Data scatter of predicted values

## Data Exploration

In [5]:
# Displaying the first 20 rows of the data
df_whatsapp_data.head(20)
# count of columns
#df_whatsapp_data.count()

,Zeitstempel,Wie alt sind Sie?,Wie ist Ihre Wohnsituation? (Wohneigentum?),Wie ist Ihre Wohnsituation? (Haus oder Wohnung?),"Wie ist Ihre Wohnsituation? (Großstadt, Stadt, Land?)",Wie ist Ihre Beschäftigung?,"Bitte schätzen Sie das monatliche Netto Gesamteinkommen in Ihrem Haushalt in Euro. (Wenn Sie dazu keine Angaben machen möchten, überspringen Sie diese Frage bitte.)",Besitzen Sie ein Auto? (auch Leasing),"Wie interessiert sind Sie an einem E-Auto als Ihr nächstes(/erstes) KFZ? (Wenn Sie generell kein Interesse am Besitz eines Autos haben, überspringen Sie diese Frage bitte)","Welchen Anteil an Ihrer Gesamtmobilität macht Ihr Auto aus? (falls Sie kein Auto besitzen, wählen Sie bitte 0% aus)",Welchen Anteil an Ihrer Gesamtmobilität macht der ÖPNV aus?,Was würde den ÖPNV für Sie attraktiver machen?,Was würde E-Autos für Sie attraktiver machen?
0,2024/04/29 4:32:21 PM OESZ,46-55,gemietet,Wohnung,Stadt,Angestellter,NaN,Auto,10,50%,0%,Besseres Netz,günstigere Preise;mehr/besser verteilte Lademö...
1,2024/04/29 4:38:37 PM OESZ,36-45,Wohneigentum,Haus,Land,Angestellter,NaN,Auto,6,90%,0%,günstigere Preise;kürzere Wartezeit zwischen z...,günstigere Preise;mehr/besser verteilte Lademö...
2,2024/04/29 4:39:08 PM OESZ,26-35,gemietet,Reihenhaus,Land,Angestellter,NaN,E-Auto,6,80%,0%,günstigere Preise,"mehr Reichweite;mehr Sicherheit (Brandgefahr, ..."
3,2024/04/29 4:39:18 PM OESZ,26-35,Wohneigentum,Haus,Land,Angestellter,7000,E-Auto,7,80%,20%,günstigere Preise;kürzere Wartezeit zwischen z...,günstigere Preise;gute Budget-E-Autos;mehr/bes...
4,2024/04/29 4:39:39 PM OESZ,36-45,gemietet,Haus,Land,Angestellter,4600,Auto,7,90%,0%,kürzere Wartezeit zwischen zwei Zügen/Bussen;b...,"mehr Reichweite;mehr Sicherheit (Brandgefahr, ..."
5,2024/04/29 4:41:40 PM OESZ,26-35,gemietet,Wohnung,Großstadt (min. 100.000 Einwohner),Angestellter,NaN,Auto,8,80%,10%,günstigere Preise,günstigere Preise;mehr/besser verteilte Lademö...
6,2024/04/29 4:44:42 PM OESZ,26-35,gemietet,Wohnung,Land,Angestellter,4200€,Auto,9,80%,20%,günstigere Preise;kürzere Wartezeit zwischen z...,mehr/besser verteilte Lademöglichkeiten;mehr R...
7,2024/04/29 4:45:26 PM OESZ,56-65,gemietet,Wohnung,Stadt,Angestellter,NaN,Auto,1,100%,0%,Keine Möglichkeit der Nutzung,mehr Reichweite
8,2024/04/29 4:45:28 PM OESZ,36-45,gemietet,Wohnung,Großstadt (min. 100.000 Einwohner),Angestellter,2700,Auto,7,40%,10%,günstigere Preise;Bessere Anbindung außerhalb ...,günstigere Preise;gute Budget-E-Autos;mehr/bes...
9,2024/04/29 4:49:05 PM OESZ,36-45,gemietet,Wohnung,Stadt,Angestellter,NaN,Auto,6,80%,10%,günstigere Preise;geringere Distanz zwischen H...,mehr/besser verteilte Lademöglichkeiten;mehr R...


## Data Cleaning

In [6]:
# Dropping duplicates and missing values
drop_data(df_whatsapp_data)
# Filling missing values with the mean
fill_data(df_whatsapp_data)
# Splitting the data into features and target
df_whatsapp_data, df_whatsapp_data_without_na, df_financial_correlation, df_mobility_correlation, df_other_correlation = split_data(df_whatsapp_data)

C:\Users\admin\AppData\Local\Temp\ipykernel_8028\1468187487.py:25: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data.fillna(method='ffill').fillna(method='bfill')


ValueError: too many values to unpack (expected 5)

## Data Preprocessing

In [ ]:
data_copy = normalise_age(df_whatsapp_data)
data_copy = normalise_monthly_income(df_whatsapp_data)
data_copy = normalise_car_ownership(df_whatsapp_data)
# Splitting the data
data, data_without_na, data_financial_correlation, data_mobility_correlation, data_living_correlation, data_age_correlation = split_data(data_copy)
# Preparing the data
financial_feature_columns = [age, monthly_income, car_ownership]
target_column = interest_e_car
X_train_f, X_test_f, y_train_f, y_test_f = prepare_data(data_without_na, financial_feature_columns, target_column)
# TODO: Implement the same for the other use cases
# Scaling the data
X_train_scaled_f, X_test_scaled_f = scale_data(X_train_f, X_test_f)

## Data Analysis

In [ ]:
# Checking the distribution of the data
df_whatsapp_data.describe()
# Analyzing the data
analyze_data(X_train_f, X_test_f, y_train_f, y_test_f)
#TODO: Implement the same for the other use cases

## Data Visualization

In [ ]:
# Visualizing the data
visualize_data(df_whatsapp_data)
# Visualizing the correlation between the features and the target
visualize_correlation(df_financial_correlation, financial_feature_columns, target_column)
#TODO: Implement the same for the other use cases

In [ ]:
# Plotting the distribution of the data
df_whatsapp_data.hist(figsize=(20, 20))
plt.show()
# add seaborn style
sns.set()
# add title and labels
plt.title('Distribution of the data')
plt.xlabel('Values')
plt.ylabel('Frequency')
plt.show()